In [1]:
%%capture
import json
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
from tqdm.notebook import tqdm
import numpy as np

# source : https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py

In [2]:
FULL_SET_PATH = './data/aan_full.json'
TEST_SET_PATH = './data/aan_test.json'
CONCAT_OUTPUT_PATH = './results/base_doc2veccosine_concat_aan.json'
SUM_OUTPUT_PATH = './results/base_doc2veccosine_sum_aan.json'

In [2]:
FULL_SET_PATH = './data/dblp_full.json'
TEST_SET_PATH = './data/dblp_test.json'
CONCAT_OUTPUT_PATH = './results/base_doc2veccosine_concat_dblp.json'
SUM_OUTPUT_PATH = './results/base_doc2veccosine_sum_dblp.json'

In [3]:
with open(FULL_SET_PATH) as f:
    full_set = json.load(f)

In [4]:
corpus = [simple_preprocess(ref['title'] + ' ' + ref['abstract']) for ref in full_set]
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(corpus)]
model = Doc2Vec(documents, vector_size=100, epochs=40, workers=4)

In [5]:
for ref in tqdm(full_set):
    document = simple_preprocess(ref['title'] + ' ' + ref['abstract'])
    ref['doc2vec_vector'] = model.infer_vector(document).reshape(1, -1)

In [6]:
full_set_dict = dict([(paper['id'], paper) for paper in full_set])

In [7]:
paper_doc2vec_vectors = [paper['doc2vec_vector'] for paper in full_set]
paper_ids = [paper['id'] for paper in full_set]

paper_doc2vec_array = np.vstack(paper_doc2vec_vectors)

In [8]:
with open(TEST_SET_PATH) as f:
    test_set = json.load(f)

# concat variant

In [9]:
input_doc2vec_vectors = []
for input_papers in tqdm(test_set):
    input_document = ''
    
    for input_paper in input_papers:
        input_document += input_paper['title'] + ' ' + input_paper['abstract']
    
    input_doc2vec_vectors.append(model.infer_vector(simple_preprocess(input_document)).reshape(1, -1))

input_doc2vec_array = np.vstack(input_doc2vec_vectors)

score_array = cosine_similarity(input_doc2vec_array, paper_doc2vec_array)

In [10]:
results = []

for input_papers, scores  in tqdm(zip(test_set, score_array.tolist())):
    result = {}
    result['input'] = [input_paper['id'] for input_paper in input_papers]
    input_paper_ids_set = set(result['input'])
    
    candidate_scores = list(zip(paper_ids, scores))     
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    filtered_candidate_scores = [cs for cs in candidate_scores if cs[0] not in input_paper_ids_set]
    
    result['output'] = [cs[0] for cs in filtered_candidate_scores[:100]]
    results.append(result)

In [11]:
with open(CONCAT_OUTPUT_PATH, 'w') as f:
    json.dump(results, f)

# sum variant

In [12]:
partial_score_arrays = []
for i in tqdm(range(3)):
    partial_input_ids = [input_papers[i]['id'] for input_papers in test_set]
    input_doc2vec_vectors = [full_set_dict[paper_id]['doc2vec_vector'] for paper_id in partial_input_ids]
    
    input_doc2vec_array = np.vstack(input_doc2vec_vectors)

    partial_score_arrays.append(cosine_similarity(input_doc2vec_array, paper_doc2vec_array))

score_array = sum(partial_score_arrays)

In [13]:
results = []

for input_papers, scores  in tqdm(zip(test_set, score_array.tolist())):
    result = {}
    result['input'] = [input_paper['id'] for input_paper in input_papers]
    input_paper_ids_set = set(result['input'])
    
    candidate_scores = list(zip(paper_ids, scores))     
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    filtered_candidate_scores = [cs for cs in candidate_scores if cs[0] not in input_paper_ids_set]
    
    result['output'] = [cs[0] for cs in filtered_candidate_scores[:100]]
    results.append(result)

In [14]:
with open(SUM_OUTPUT_PATH, 'w') as f:
    json.dump(results, f)